In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    sum, avg, approx_count_distinct, cos, sqrt
)
import pandas as pd
import os

# Initialize Spark session
spark = SparkSession.builder.appName("AggregationStandalone").getOrCreate()

In [ ]:
# Sample dataset to simulate BedBricks events
sample_data = [
    {"event_name": "purchase", "geo": {"state": "CA", "city": "San Francisco"}, "ecommerce": {"purchase_revenue_in_usd": 100.0, "total_item_quantity": 2}, "user_id": 1},
    {"event_name": "purchase", "geo": {"state": "CA", "city": "San Francisco"}, "ecommerce": {"purchase_revenue_in_usd": 150.0, "total_item_quantity": 3}, "user_id": 2},
    {"event_name": "click", "geo": {"state": "CA", "city": "Los Angeles"}, "ecommerce": {"purchase_revenue_in_usd": 0.0, "total_item_quantity": 0}, "user_id": 3},
    {"event_name": "purchase", "geo": {"state": "NY", "city": "New York"}, "ecommerce": {"purchase_revenue_in_usd": 200.0, "total_item_quantity": 4}, "user_id": 4},
    {"event_name": "click", "geo": {"state": "NY", "city": "Buffalo"}, "ecommerce": {"purchase_revenue_in_usd": 0.0, "total_item_quantity": 0}, "user_id": 5},
    {"event_name": "purchase", "geo": {"state": "NY", "city": "New York"}, "ecommerce": {"purchase_revenue_in_usd": 120.0, "total_item_quantity": 2}, "user_id": 6},
]

In [ ]:
# Normalize nested dictionaries and convert to Spark DataFrame
df_pd = pd.json_normalize(sample_data)
df = spark.createDataFrame(df_pd)

# Display original data
print("Original DataFrame")
df.show()

In [ ]:
# Grouping Examples
print("Group by event_name")
df.groupBy("event_name").count().show()

print("Group by geo.state, geo.city")
df.groupBy("geo.state", "geo.city").count().show()

In [ ]:
# Aggregation Examples
print("Average purchase revenue per state")
df.groupBy("geo.state").avg("ecommerce.purchase_revenue_in_usd").show()

print("Total quantity and sum of revenue by state and city")
df.groupBy("geo.state", "geo.city") \
  .sum("ecommerce.total_item_quantity", "ecommerce.purchase_revenue_in_usd").show()

In [ ]:
print("Sum with alias")
df.groupBy("geo.state").agg(sum("ecommerce.total_item_quantity").alias("total_purchases")).show()

In [ ]:
print("Multiple aggregates")
df.groupBy("geo.state").agg(
    avg("ecommerce.total_item_quantity").alias("avg_quantity"),
    approx_count_distinct("user_id").alias("distinct_users")
).show()

In [ ]:
# Math functions example
print("Math operations using built-in functions")
spark.range(10).withColumn("sqrt", sqrt("id")).withColumn("cos", cos("id")).show()

In [ ]:
# Stop Spark session
spark.stop()